In [1]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [2]:
nb_classes = 10
batch_size = 128
maxepoches = 250
learning_rate = 0.1
lr_decay = 1e-6
lr_drop = 20
def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [3]:
#Loading and splitting the dataset into train, validation and test


(X_Train, y_Train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

In [4]:
X_Train.shape

(50000, 32, 32, 3)

In [5]:
# Reshape them to batch_size, width,height,#channels
X_train = X_train.reshape(40000, 32, 32, 3)
X_val = X_val.reshape(10000, 32, 32, 3)
X_test = X_test.reshape(10000, 32, 32, 3)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255
X_test /= 255

In [6]:
init=RandomNormal(mean=0,stddev=0.02)
input_shape = (32, 32, 3) # Input shape of each image
weight_decay = 0.0005
def build_model():
    # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',
                     input_shape=input_shape,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256,kernel_regularizer=regularizers.l2(weight_decay), name='dense_1'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(10, name='dense_2'))
    model.add(Activation('softmax'))
    return model
teacher = build_model()

sgd = SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)

teacher.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])


In [7]:
# teacher.fit(X_train,Y_train,batch_size=128,epochs=150,verbose=1,callbacks=[reduce_lr],validation_data=(X_val,Y_val))

In [8]:
teacher.load_weights("Cifar10_Teacher.h5")

In [9]:
# Y_test = np_utils.to_categorical(y_test, nb_classes)
loss, acc =teacher.evaluate(X_test, y_test, verbose=1)
loss, acc

313/313 [==============================] - 5s 8ms/step - loss: 0.8247 - accuracy: 0.8996


(0.834343671798706, 0.8992000222206116)

In [10]:
#Collect the dense vector from the previous layer output and store it in a different model
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)

In [11]:
#Extracting dense representation from the teacher network
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)

In [24]:
#Splitting the training dense vector among N students(in this case 2)
s1Train=train_dense[:,:32]
s2Train=train_dense[:,32:64]
s3Train=train_dense[:,64:96]
s4Train=train_dense[:,96:128]

s5Train=train_dense[:,128:160]
s6Train=train_dense[:,160:192]
s7Train=train_dense[:,192:224]
s8Train=train_dense[:,224:256]

s1Val=val_dense[:,:32]
s2Val=val_dense[:,32:64]
s3Val=val_dense[:,64:96]
s4Val=val_dense[:,96:128]

s5Val=val_dense[:,128:160]
s6Val=val_dense[:,160:192]
s7Val=val_dense[:,192:224]
s8Val=val_dense[:,224:256]

In [16]:
def define_model(name):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3), name=name))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform',name='req'+name))

    model.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
    return model

student1 = define_model('s1')
student1.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 8, 8, 64)        

TCN


In [17]:
# Testing with LR
s1=define_model("s1")
s2=define_model("s2")
s3=define_model("s3")
s4=define_model("s4")
s5=define_model("s5")
s6=define_model("s6")
s7=define_model("s7")
s8=define_model("s8")

opt=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, amsgrad=False)
s1.compile(loss='mse', optimizer=opt)
s2.compile(loss='mse', optimizer=opt)
s3.compile(loss='mse', optimizer=opt)
s4.compile(loss='mse', optimizer=opt)
s5.compile(loss='mse', optimizer=opt)
s6.compile(loss='mse', optimizer=opt)
s7.compile(loss='mse', optimizer=opt)
s8.compile(loss='mse', optimizer=opt)

In [18]:
s1.fit(X_train,s1Train,
          batch_size=256,
          epochs=80,
          verbose=1,
          validation_data=(X_val,s1Val))

s2.fit(X_train,s2Train,
          batch_size=256,
          epochs=60,
          verbose=1,
          validation_data=(X_val,s2Val))

Epoch 1/80
157/157 [==============================] - 3s 15ms/step - loss: 2.1102 - val_loss: 1.7533
Epoch 2/80
157/157 [==============================] - 2s 13ms/step - loss: 1.7536 - val_loss: 1.7478
Epoch 3/80
157/157 [==============================] - 2s 13ms/step - loss: 1.7482 - val_loss: 1.7373
Epoch 4/80
157/157 [==============================] - 2s 15ms/step - loss: 1.7438 - val_loss: 1.7228
Epoch 5/80
157/157 [==============================] - 2s 13ms/step - loss: 1.7156 - val_loss: 1.7093
Epoch 6/80
157/157 [==============================] - 2s 16ms/step - loss: 1.7011 - val_loss: 1.6979
Epoch 7/80
157/157 [==============================] - 3s 17ms/step - loss: 1.6939 - val_loss: 1.6883
Epoch 8/80
157/157 [==============================] - 3s 17ms/step - loss: 1.6870 - val_loss: 1.6783
Epoch 9/80
157/157 [==============================] - 3s 16ms/step - loss: 1.6765 - val_loss: 1.6659
Epoch 10/80
157/157 [==============================] - 2s 15ms/step - loss: 1.6548 - val_lo

157/157 [==============================] - 2s 14ms/step - loss: 1.7337 - val_loss: 1.5810
Epoch 2/60
157/157 [==============================] - 2s 16ms/step - loss: 1.5579 - val_loss: 1.5459
Epoch 3/60
157/157 [==============================] - 3s 16ms/step - loss: 1.5288 - val_loss: 1.5099
Epoch 4/60
157/157 [==============================] - 3s 18ms/step - loss: 1.5047 - val_loss: 1.4863
Epoch 5/60
157/157 [==============================] - 3s 17ms/step - loss: 1.4892 - val_loss: 1.4739
Epoch 6/60
157/157 [==============================] - 3s 18ms/step - loss: 1.4773 - val_loss: 1.4750
Epoch 7/60
157/157 [==============================] - 3s 18ms/step - loss: 1.4711 - val_loss: 1.4676
Epoch 8/60
157/157 [==============================] - 3s 17ms/step - loss: 1.4688 - val_loss: 1.4643
Epoch 9/60
157/157 [==============================] - 3s 18ms/step - loss: 1.4606 - val_loss: 1.4610
Epoch 10/60
157/157 [==============================] - 3s 18ms/step - loss: 1.4578 - val_loss: 1.4601


In [25]:
s3.fit(X_train,s3Train,
          batch_size=256,
          epochs=70,
          verbose=1,
          validation_data=(X_val,s3Val))

s4.fit(X_train,s4Train,
          batch_size=256,
          epochs=60,
          verbose=1,
          validation_data=(X_val,s4Val))

Epoch 1/60
157/157 [==============================] - 2s 13ms/step - loss: 1.5610 - val_loss: 1.5579
Epoch 2/60
157/157 [==============================] - 2s 14ms/step - loss: 1.5587 - val_loss: 1.5558
Epoch 3/60
157/157 [==============================] - 2s 13ms/step - loss: 1.5568 - val_loss: 1.5541
Epoch 4/60
157/157 [==============================] - 2s 15ms/step - loss: 1.5551 - val_loss: 1.5527
Epoch 5/60
157/157 [==============================] - 2s 14ms/step - loss: 1.5536 - val_loss: 1.5513
Epoch 6/60
157/157 [==============================] - 2s 13ms/step - loss: 1.5524 - val_loss: 1.5502
Epoch 7/60
157/157 [==============================] - 2s 15ms/step - loss: 1.5512 - val_loss: 1.5491
Epoch 8/60
157/157 [==============================] - 2s 14ms/step - loss: 1.5501 - val_loss: 1.5481
Epoch 9/60
157/157 [==============================] - 2s 14ms/step - loss: 1.5491 - val_loss: 1.5472
Epoch 10/60
157/157 [==============================] - 2s 15ms/step - loss: 1.5483 - val_lo

In [26]:
s5.fit(X_train,s5Train,
          batch_size=256,
          epochs=70,
          verbose=1,
          validation_data=(X_val,s5Val))

s6.fit(X_train,s6Train,
          batch_size=256,
          epochs=60,
          verbose=1,
          validation_data=(X_val,s6Val))
s7.fit(X_train,s7Train,
          batch_size=256,
          epochs=70,
          verbose=1,
          validation_data=(X_val,s7Val))

s8.fit(X_train,s8Train,
          batch_size=256,
          epochs=60,
          verbose=1,
          validation_data=(X_val,s8Val))

Epoch 1/70
157/157 [==============================] - 2s 13ms/step - loss: 1.7862 - val_loss: 1.5651
Epoch 2/70
157/157 [==============================] - 3s 18ms/step - loss: 1.5543 - val_loss: 1.5226
Epoch 3/70
157/157 [==============================] - 2s 15ms/step - loss: 1.5185 - val_loss: 1.4713
Epoch 4/70
157/157 [==============================] - 2s 14ms/step - loss: 1.4851 - val_loss: 1.4441
Epoch 5/70
157/157 [==============================] - 2s 14ms/step - loss: 1.4678 - val_loss: 1.4380
Epoch 6/70
157/157 [==============================] - 3s 16ms/step - loss: 1.4483 - val_loss: 1.4169
Epoch 7/70
157/157 [==============================] - 2s 13ms/step - loss: 1.4371 - val_loss: 1.4044
Epoch 8/70
157/157 [==============================] - 2s 13ms/step - loss: 1.4230 - val_loss: 1.3864
Epoch 9/70
157/157 [==============================] - 3s 18ms/step - loss: 1.4163 - val_loss: 1.3746
Epoch 10/70
157/157 [==============================] - 3s 18ms/step - loss: 1.4027 - val_lo

157/157 [==============================] - 2s 16ms/step - loss: 1.2792 - val_loss: 1.2490
Epoch 12/60
157/157 [==============================] - 2s 15ms/step - loss: 1.2692 - val_loss: 1.2397
Epoch 13/60
157/157 [==============================] - 2s 15ms/step - loss: 1.2606 - val_loss: 1.2308
Epoch 14/60
157/157 [==============================] - 2s 12ms/step - loss: 1.2581 - val_loss: 1.2277
Epoch 15/60
157/157 [==============================] - 3s 18ms/step - loss: 1.2519 - val_loss: 1.2279
Epoch 16/60
157/157 [==============================] - 3s 18ms/step - loss: 1.2492 - val_loss: 1.2128
Epoch 17/60
157/157 [==============================] - 3s 18ms/step - loss: 1.2398 - val_loss: 1.2082
Epoch 18/60
157/157 [==============================] - 3s 18ms/step - loss: 1.2403 - val_loss: 1.2063
Epoch 19/60
157/157 [==============================] - 3s 16ms/step - loss: 1.2387 - val_loss: 1.1988
Epoch 20/60
157/157 [==============================] - 2s 16ms/step - loss: 1.2261 - val_loss:

157/157 [==============================] - 3s 17ms/step - loss: 1.4769 - val_loss: 1.4776
Epoch 32/70
157/157 [==============================] - 3s 18ms/step - loss: 1.4774 - val_loss: 1.4776
Epoch 33/70
157/157 [==============================] - 3s 18ms/step - loss: 1.4755 - val_loss: 1.4776
Epoch 34/70
157/157 [==============================] - 3s 18ms/step - loss: 1.4789 - val_loss: 1.4776
Epoch 35/70
157/157 [==============================] - 3s 18ms/step - loss: 1.4810 - val_loss: 1.4776
Epoch 36/70
157/157 [==============================] - 3s 17ms/step - loss: 1.4727 - val_loss: 1.4776
Epoch 37/70
157/157 [==============================] - 3s 17ms/step - loss: 1.4832 - val_loss: 1.4776
Epoch 38/70
157/157 [==============================] - 2s 15ms/step - loss: 1.4752 - val_loss: 1.4776
Epoch 39/70
157/157 [==============================] - 3s 16ms/step - loss: 1.4784 - val_loss: 1.4776
Epoch 40/70
157/157 [==============================] - 3s 17ms/step - loss: 1.4789 - val_loss:

157/157 [==============================] - 3s 16ms/step - loss: 1.1091 - val_loss: 1.0400
Epoch 42/60
157/157 [==============================] - 3s 17ms/step - loss: 1.0971 - val_loss: 1.0322
Epoch 43/60
157/157 [==============================] - 3s 17ms/step - loss: 1.0933 - val_loss: 1.0308
Epoch 44/60
157/157 [==============================] - 3s 16ms/step - loss: 1.0844 - val_loss: 1.0066
Epoch 45/60
157/157 [==============================] - 2s 15ms/step - loss: 1.0805 - val_loss: 0.9987
Epoch 46/60
157/157 [==============================] - 3s 17ms/step - loss: 1.0675 - val_loss: 0.9997
Epoch 47/60
157/157 [==============================] - 2s 14ms/step - loss: 1.0657 - val_loss: 0.9931
Epoch 48/60
157/157 [==============================] - 2s 14ms/step - loss: 1.0596 - val_loss: 0.9944
Epoch 49/60
157/157 [==============================] - 2s 15ms/step - loss: 1.0529 - val_loss: 0.9778
Epoch 50/60
157/157 [==============================] - 3s 16ms/step - loss: 1.0481 - val_loss:

In [27]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
o3=s3.get_layer("reqs3").output
o4=s4.get_layer("reqs4").output
o5=s5.get_layer("reqs5").output
o6=s6.get_layer("reqs6").output
o7=s7.get_layer("reqs7").output
o8=s8.get_layer("reqs8").output
output=tensorflow.keras.layers.concatenate([o1,o2,o3,o4,o5,o6,o7,o8])

output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)

mm8=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,
           s3.get_layer("s3").input,s4.get_layer("s4").input,
           s5.get_layer("s5").input,s6.get_layer("s6").input,
           s7.get_layer("s7").input,s8.get_layer("s8").input], output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm8.get_layer('d1').set_weights(my_weights)

In [28]:
i=0
for l in mm8.layers[:len(mm8.layers)-2]:
    l.trainable=False
#     print(l)

mm8.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0002),
              metrics=['accuracy'])

# Without finetune
batch_size = 256
mm8_history=mm8.fit([X_train,X_train,X_train,X_train,X_train,X_train,X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=1,
          validation_data=([X_val,X_val,X_val,X_val,X_val,X_val,X_val,X_val], Y_val))

Epoch 1/20
157/157 [==============================] - 7s 37ms/step - loss: 1.2816 - accuracy: 0.6549 - val_loss: 0.7401 - val_accuracy: 0.7840
Epoch 2/20
157/157 [==============================] - 5s 31ms/step - loss: 0.8359 - accuracy: 0.7529 - val_loss: 0.6429 - val_accuracy: 0.8078
Epoch 3/20
157/157 [==============================] - 5s 31ms/step - loss: 0.7282 - accuracy: 0.7824 - val_loss: 0.6218 - val_accuracy: 0.8126
Epoch 4/20
157/157 [==============================] - 5s 31ms/step - loss: 0.6965 - accuracy: 0.7911 - val_loss: 0.6176 - val_accuracy: 0.8124
Epoch 5/20
157/157 [==============================] - 5s 31ms/step - loss: 0.6899 - accuracy: 0.7976 - val_loss: 0.6141 - val_accuracy: 0.8136
Epoch 6/20
157/157 [==============================] - 5s 31ms/step - loss: 0.6773 - accuracy: 0.8001 - val_loss: 0.6117 - val_accuracy: 0.8133
Epoch 7/20
157/157 [==============================] - 5s 31ms/step - loss: 0.6686 - accuracy: 0.8004 - val_loss: 0.6093 - val_accuracy: 0.8133